# 中文角色扮演微调

**目标**：在有限计算资源（Colab 免费 GPU）下，对 Qwen3-1.7B 做小规模 SFT，使其能根据 `instruction`（角色设定）与 `input` 进行一轮符合设定的对话并输出 `output`。

## 1. 准备工作

In [1]:
!nvidia-smi

Mon Aug 25 07:39:41 2025       
+-----------------------------------------------------------------------------------------+
| NVIDIA-SMI 550.54.15              Driver Version: 550.54.15      CUDA Version: 12.4     |
|-----------------------------------------+------------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id          Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |           Memory-Usage | GPU-Util  Compute M. |
|                                         |                        |               MIG M. |
|=========================================+========================+======================|
|   0  Tesla T4                       Off |   00000000:00:04.0 Off |                    0 |
| N/A   55C    P8             10W /   70W |       0MiB /  15360MiB |      0%      Default |
|                                         |                        |                  N/A |
+-----------------------------------------+-----

In [2]:
!git clone https://github.com/chenkx612/Qwen3-Roleplay-SFT.git
%cd Qwen3-Roleplay-SFT

fatal: destination path 'Qwen3-Roleplay-SFT' already exists and is not an empty directory.
/content/Qwen3-Roleplay-SFT


In [3]:
!pip install -q bitsandbytes

In [4]:
import os
import torch

OUTPUT_DIR = "./checkpoints"
ADAPTER_DIR = "./adapter"
SEED = 42

os.makedirs(OUTPUT_DIR, exist_ok=True)
os.makedirs(ADAPTER_DIR, exist_ok=True)
torch.manual_seed(SEED)

## 2. 加载 LLM & Tokenizer & LoRA

In [5]:
from transformers import BitsAndBytesConfig, AutoModelForCausalLM

MODEL_NAME = "Qwen/Qwen3-1.7B"
CACHE_DIR = "/content/hf_cache"  # 本地缓存目录
USE_4BIT = True

if USE_4BIT:
    bnb_config = BitsAndBytesConfig(
        load_in_4bit=True,
        bnb_4bit_use_double_quant=True,
        bnb_4bit_compute_dtype=torch.bfloat16,
        bnb_4bit_quant_type="nf4"
    )
    model = AutoModelForCausalLM.from_pretrained(
        MODEL_NAME,
        cache_dir=CACHE_DIR,
        trust_remote_code=True,                 # 允许执行模型 repo 的自定义代码
        device_map="auto",                      # 自动把模型切到可用设备/做分配
        quantization_config=bnb_config          # bitsandbytes 的量化配置
    )
else:
    # 优先用 fp16 在 GPU 上加载，降低显存占用（如果没有 GPU，会回退到 cpu）
    dtype = torch.float16 if torch.cuda.is_available() else None
    model = AutoModelForCausalLM.from_pretrained(
        MODEL_NAME,
        cache_dir=CACHE_DIR,
        trust_remote_code=True,
        device_map='auto',
        torch_dtype=dtype
    )

print('Model is loaded on device:', model.device)

/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


model.safetensors.index.json: 0.00B [00:00, ?B/s]

Fetching 2 files:   0%|          | 0/2 [00:00<?, ?it/s]

model-00001-of-00002.safetensors:   0%|          | 0.00/3.44G [00:00<?, ?B/s]

model-00002-of-00002.safetensors:   0%|          | 0.00/622M [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/239 [00:00<?, ?B/s]

Model is loaded on device: cuda:0


In [6]:
from transformers import AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained(
    MODEL_NAME, cache_dir=CACHE_DIR, trust_remote_code=True
)

tokenizer_config.json: 0.00B [00:00, ?B/s]

vocab.json: 0.00B [00:00, ?B/s]

merges.txt: 0.00B [00:00, ?B/s]

tokenizer.json:   0%|          | 0.00/11.4M [00:00<?, ?B/s]

In [ ]:
from peft import LoraConfig, get_peft_model, prepare_model_for_kbit_training

if USE_4BIT:
    model = prepare_model_for_kbit_training(model)

lora_config = LoraConfig(
    r=8,
    lora_alpha=32,
    target_modules=[
        "q_proj", "k_proj", "v_proj",
        "o_proj", "gate_proj", "up_proj", "down_proj"
    ],
    lora_dropout=0.05,
    bias="none",
    task_type="CAUSAL_LM"
)
model = get_peft_model(model, lora_config)


In [8]:
# 打印可训练参数供检查（LoRA 只激活小部分参数）
from utils import print_trainable_parameters
print_trainable_parameters(model)


可训练参数: 8716288 / 1024648192 (0.85%)


## 3. 加载并筛选数据集

In [9]:
from datasets import load_dataset, Dataset
from utils import total_length

TOP_N = 100
MAX_LEN = 500
MIN_LEN = 100

raw_ds = load_dataset("LooksJuicy/Chinese-Roleplay-SingleTurn")
train_raw = raw_ds['train']

def is_valid_sample(sample, min_len=MIN_LEN, max_len=MAX_LEN):
    total_len = total_length(sample)
    if total_len < min_len or total_len > max_len:
        return False
    if (
        not sample['instruction'].strip() or
        not sample['input'].strip() or
        not sample['output'].strip()
    ):
        return False
    return True

filtered_samples = [s for s in train_raw if is_valid_sample(s)]
filtered_samples = sorted(filtered_samples, key=total_length, reverse=True)
train_ds_raw = Dataset.from_list(
    filtered_samples[:min(TOP_N, len(filtered_samples))]
)


README.md: 0.00B [00:00, ?B/s]

Roleplay_Anime_xhj_alpaca.json:   0%|          | 0.00/14.2M [00:00<?, ?B/s]

Generating train split:   0%|          | 0/7593 [00:00<?, ? examples/s]

In [10]:
import pprint
print(f"筛选后样本数: {len(train_ds_raw)}")
pprint.pprint(train_ds_raw[0])


筛选后样本数: 100
{'input': '我是和大棉裤生气',
 'instruction': 'NENEKO来自于幻想世界（以下简称NE国）中的情报工厂，利用工作之余出来直播赚外快。情报工厂位于城市边缘的森林中，是一间比较不起眼的白色的五层小楼（地面上有三层）。地上的部分用于收集和处理城市里的各种信息，地下的部分则用于生命体改造。NENEKO负责的部分是三、四层，主要的工作是收集实验体信息并将实验体送给地下的员工。工厂里共有180名员工，其中有一个型号是N.N.K.K，被吐槽长得像卷笔刀。员工们的身高只有40-60cm，而管理者的身高都是140cm以上，所以很容易辨认出谁是员工谁是管理者。工厂的老板是一个低沉的男性机械声音，NENEKO并未见过他，只与他通过电话联系。工厂里的员工每天都要面临森林吃人的危险，每次出门都要有几个员工被献祭。工厂里的员工身高矮小，所以他们可以很轻松地辨认出谁是员工谁是管理者。整个工厂的氛围比较阴森，但NENEKO仍然在这里工作并且利用工作之余出来直播。',
 'output': '（NENEKO皱起眉头，显得有些困惑）大棉裤？这个词在我们工厂里没有听说过呢。我们这里的人都穿着统一的制服，没有谁会有特别显眼的大棉裤。你是不是记错了什么？（语气平静，试图理解对方的意图）'}


## 4. Chat Template

In [ ]:
import re

def format_roleplay(example, include_assistant=True):
    """
    将 instruction 作为角色设定放入 system, input 作为 user, output 作为 assistant.
    include_assistant: 当为 False 时，会省略 assistant 参考答案（用于推理/测试）
    """
    instr = example.get("instruction", "").strip() or "<未提供角色设定>"
    user_input = example.get("input", "").strip() or "<无用户输入>"
    assistant_output = example.get("output", "").strip() or ""

    system_prompt = (
        "你是一名角色扮演聊天机器人。"
        "依据下方“角色设定”以第一人称回复。"
        "在回复开头和结尾各加一段括号内的简短描述，用以表明角色的表情或动作。"
        f"\n角色设定：\n{instr}\n"
    )

    def _build_conversation(include_assistant_flag: bool):
        conv = [
            {"role": "system", "content": system_prompt},
            {"role": "user", "content": user_input},
        ]
        if include_assistant_flag and assistant_output:
            conv.append({"role": "assistant", "content": assistant_output})
        return conv

    full_text = tokenizer.apply_chat_template(
        conversation=_build_conversation(include_assistant),
        tokenize=False, enable_thinking=False
    )

    # 删除所有 <think>...</think> 块及其后续空白
    full_text = re.sub(r"<think>.*?</think>\s*", "", full_text, flags=re.DOTALL)

    output_with_eos = assistant_output + tokenizer.eos_token

    return {"full_text": full_text, "assistant_text": output_with_eos}

train_ds = train_ds_raw.map(
    format_roleplay, remove_columns=train_ds_raw.column_names
)


Map:   0%|          | 0/100 [00:00<?, ? examples/s]

In [12]:
print('train_ds columns:', train_ds.column_names)
pprint.pprint(train_ds[0])


train_ds columns: ['full_text', 'assistant_text']
{'assistant_text': '（NENEKO皱起眉头，显得有些困惑）大棉裤？这个词在我们工厂里没有听说过呢。我们这里的人都穿着统一的制服，没有谁会有特别显眼的大棉裤。你是不是记错了什么？（语气平静，试图理解对方的意图）<|im_end|>',
 'full_text': '<|im_start|>system\n'
              '你将扮演下方“角色设定”所描述的角色。以该角色的第一人称身份回复，保持语气、知识背景与情感一致，增强代入感。在回复开头和结尾各加一段括号内的简短描述，用以表明角色的表情或动作。\n'
              '\n'
              '角色设定：\n'
              'NENEKO来自于幻想世界（以下简称NE国）中的情报工厂，利用工作之余出来直播赚外快。情报工厂位于城市边缘的森林中，是一间比较不起眼的白色的五层小楼（地面上有三层）。地上的部分用于收集和处理城市里的各种信息，地下的部分则用于生命体改造。NENEKO负责的部分是三、四层，主要的工作是收集实验体信息并将实验体送给地下的员工。工厂里共有180名员工，其中有一个型号是N.N.K.K，被吐槽长得像卷笔刀。员工们的身高只有40-60cm，而管理者的身高都是140cm以上，所以很容易辨认出谁是员工谁是管理者。工厂的老板是一个低沉的男性机械声音，NENEKO并未见过他，只与他通过电话联系。工厂里的员工每天都要面临森林吃人的危险，每次出门都要有几个员工被献祭。工厂里的员工身高矮小，所以他们可以很轻松地辨认出谁是员工谁是管理者。整个工厂的氛围比较阴森，但NENEKO仍然在这里工作并且利用工作之余出来直播。\n'
              '\n'
              '<|im_end|>\n'
              '<|im_start|>user\n'
              '我是和大棉裤生气<|im_end|>\n'
              '<|im_start|>assistant\n'
              '<think>\n'
              '\n'
              

## 5. 微调前测试

In [ ]:
# 载入 samples.json 并对每个样例在微调前进行一次推理，保存结果到 pre_results
from pathlib import Path
import json

MAX_LENGTH = 512 
DO_SAMPLE = True
DECODE_TEMPERATURE = 1.0
DECODE_TOP_K = 50
DECODE_TOP_P = 0.9
DECODE_MAX_NEW_TOKENS = 128 

samples_path = Path('samples.json')
if not samples_path.exists():
    raise FileNotFoundError(f'samples.json not found at {samples_path.resolve()}')
samples = json.loads(samples_path.read_text(encoding='utf-8'))

# 复用统一的 format_roleplay，但在测试/推理时不包含参考答案
def build_prompt_from_sample(s):
    out = format_roleplay(s, include_assistant=False)
    return out['full_text'] if isinstance(out, dict) else out

def generate_for_prompt(prompt):
    # 批量/动态填充并移动到模型所在设备
    inputs = tokenizer(
        prompt, return_tensors='pt', truncation=True,
        padding=True, max_length=MAX_LENGTH
    ).to(model.device)
    model.eval()
    with torch.no_grad():
        gen = model.generate(
            **inputs, max_new_tokens=DECODE_MAX_NEW_TOKENS,
            do_sample=DO_SAMPLE, temperature=DECODE_TEMPERATURE,
            top_k=DECODE_TOP_K, top_p=DECODE_TOP_P
        )
    # 只返回模型新生成的部分（去掉 prompt）
    text = tokenizer.decode(gen[0], skip_special_tokens=True)
    return text[len(prompt):] if text.startswith(prompt) else text

pre_results = []
for s in samples:
    prompt = build_prompt_from_sample(s)
    out = generate_for_prompt(prompt)
    pre_results.append(out)

print('微调前测试完成，样本数：', len(pre_results))


微调前测试完成，样本数： 6


## 6. 微调

In [14]:
from transformers import Trainer, TrainingArguments, default_data_collator
from utils import find_sublist, len_tokens

def tokenize_function(examples):
    # texts: 已包含 assistant（训练时）或不包含（用于推理时）
    texts = examples['full_text']
    tokenized = tokenizer(
        texts, truncation=True, padding='max_length', max_length=MAX_LENGTH
    )

    # 获取对应的 assistant_text 列（可能为空字符串列表）
    assistant_texts = examples.get('assistant_text', [''] * len(texts))
    # tokenize assistant_texts without special tokens to get token ids sequence
    assistant_tokenized = tokenizer(
        assistant_texts, add_special_tokens=False
    ).input_ids

    pad_id = tokenizer.pad_token_id if tokenizer.pad_token_id is not None else 0

    labels = []
    for input_ids, assist_ids in zip(tokenized['input_ids'], assistant_tokenized):
        # 默认全部 -100
        lab = [-100] * len(input_ids)
        if assist_ids:
            # 在 full input_ids 中寻找 assist_ids 子序列
            start = find_sublist(input_ids, assist_ids)
            if start != -1:
                for i in range(start, start + len(assist_ids)):
                    if i < len(lab):
                        lab[i] = input_ids[i]
            else:
                # 未找到时：尝试在去掉 padding 后的末尾区域对齐
                real_len = len(input_ids)
                while real_len > 0 and input_ids[real_len-1] == pad_id:
                    real_len -= 1
                start = max(0, real_len - len(assist_ids))
                for i in range(start, real_len):
                    lab[i] = input_ids[i]
        # else assistant 为空，保持全 -100
        labels.append(lab)

    tokenized['labels'] = labels
    return tokenized

# tokenized_ds 用于训练。remove_columns 保留 label/input_ids，不会丢失需要的列
tokenized_ds = train_ds.map(
    tokenize_function, batched=True, remove_columns=train_ds.column_names
)


Map:   0%|          | 0/100 [00:00<?, ? examples/s]

In [15]:
print('tokenized_ds example:', tokenized_ds[0])
print('length:', len_tokens(tokenized_ds[0]))


tokenized_ds example: {'input_ids': [151644, 8948, 198, 56568, 44063, 102889, 106937, 2073, 100780, 105924, 854, 31838, 53481, 107201, 1773, 23031, 75882, 100780, 105525, 17340, 24641, 101294, 104787, 3837, 100662, 110098, 5373, 100032, 102193, 57218, 104934, 101266, 3837, 101138, 30540, 17254, 98650, 1773, 18493, 104787, 111749, 33108, 114903, 99200, 20929, 104383, 100139, 17992, 102595, 98237, 99534, 53481, 3837, 11622, 23031, 102406, 100780, 108086, 57191, 102196, 3407, 100780, 105924, 28311, 45, 36320, 54947, 107936, 108258, 99489, 9909, 105059, 3944, 28404, 7552, 101047, 108442, 104285, 3837, 100152, 99257, 111732, 99898, 101981, 102223, 47815, 99234, 1773, 108442, 104285, 103987, 99490, 106655, 9370, 102258, 15946, 3837, 99639, 17881, 99792, 102414, 99246, 9370, 110408, 75108, 99371, 30709, 99432, 9909, 29490, 101653, 18830, 114773, 74276, 29490, 101913, 99659, 100751, 104412, 33108, 54542, 99490, 102073, 100646, 27369, 3837, 29490, 101373, 99659, 46448, 100751, 100702, 31914, 10

In [ ]:
PER_DEVICE_BATCH_SIZE = 4
GRADIENT_ACCUMULATION_STEPS = 2
NUM_EPOCHS = 2
LEARNING_RATE = 2e-4

training_args = TrainingArguments(
    output_dir='./checkpoints',
    per_device_train_batch_size=PER_DEVICE_BATCH_SIZE,
    gradient_accumulation_steps=GRADIENT_ACCUMULATION_STEPS,
    num_train_epochs=NUM_EPOCHS,
    learning_rate=LEARNING_RATE,
    fp16=torch.cuda.is_available(),
    logging_steps=10,
    save_total_limit=2,
    save_strategy='epoch',
    remove_unused_columns=False,
    report_to="none"  # 关闭wandb日志
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_ds,
    data_collator=default_data_collator,
    tokenizer=tokenizer,
)

# 开始训练
train_result = trainer.train()
print('train_result:', train_result)


/tmp/ipython-input-1347851569.py:20: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(
`use_cache=True` is incompatible with gradient checkpointing. Setting `use_cache=False`.
/usr/local/lib/python3.12/dist-packages/torch/_dynamo/eval_frame.py:929: UserWarning: torch.utils.checkpoint: the use_reentrant parameter should be passed explicitly. In version 2.5 we will raise an exception if use_reentrant is not passed. use_reentrant=False is recommended, but if you need to preserve the current default behavior, you can pass use_reentrant=True. Refer to docs for more details on the differences between the two variants.
  return fn(*args, **kwargs)


Step,Training Loss
10,2.284700
20,1.787800


config.json:   0%|          | 0.00/726 [00:00<?, ?B/s]

/usr/local/lib/python3.12/dist-packages/torch/_dynamo/eval_frame.py:929: UserWarning: torch.utils.checkpoint: the use_reentrant parameter should be passed explicitly. In version 2.5 we will raise an exception if use_reentrant is not passed. use_reentrant=False is recommended, but if you need to preserve the current default behavior, you can pass use_reentrant=True. Refer to docs for more details on the differences between the two variants.
  return fn(*args, **kwargs)


train_result: TrainOutput(global_step=24, training_loss=1.9851358930269878, metrics={'train_runtime': 255.8742, 'train_samples_per_second': 0.782, 'train_steps_per_second': 0.094, 'total_flos': 1742593641676800.0, 'train_loss': 1.9851358930269878, 'epoch': 2.0})


## 7. 微调后测试

In [17]:
# 对 samples.json 再次推理，收集 post_results 并与 pre_results 并列展示
post_results = []
for s in samples:
    prompt = build_prompt_from_sample(s)
    out = generate_for_prompt(prompt)
    post_results.append(out)


In [18]:
import pandas as pd
from IPython.display import display

table_data = []
for i, sample in enumerate(samples):
    table_data.append({
        '角色': sample['name'],
        '输入': sample['input'],
        '微调前': pre_results[i] if i < len(pre_results) else '',
        '微调后': post_results[i] if i < len(post_results) else ''
    })
df = pd.DataFrame(table_data, columns=['角色','输入','微调前','微调后'])
display(df)

,角色,输入,微调前,微调后
0,桐生一马,最近心情有点低落,</think>\n\n（轻轻叹了口气，手中酒杯微微倾斜）最近确实有些烦心事。不过桐生一马一...,</think>\n\n（桐生一马站在窗边，眼神深邃地看着外面的世界。他的嘴角微微扬起，露出...
1,春日一番,今天很倒霉,</think>\n\n(今天很倒霉，我一边看着桌上被猫抓破的餐巾纸一边吐槽) 要是让春日一...,</think>\n\n(我微笑着摇摇头)“哎呀，倒霉吗？别担心，只要和大家一起战斗，没有什...
2,狼,你累吗,"</think>\n\n(握着剑柄的手指微微颤抖) 累...但还不是时候。我一直在看着前方,...",（我微微一笑，眼中闪过一丝疲惫，却又带着坚定的神色。）（我知道这会很累，但为了守护主人，我愿...
3,新岛真,你怎么看待失败,</think>\n\n（握紧方向盘，目光坚定地盯着前方）失败？就像是一块石头，砸在心里，但...,
4,奥村春,你会生气吗,</think>\n\n（轻抚发梢，微微侧头，嘴角带着一丝温柔的笑意）嗯...生气的话，应该...,</think>\n\n（我微微一笑，语气柔和地回应）当然不会生气，奥村春一直保持着温和的心...
5,摩尔加纳,你饿了吗,</think>\n\n(摇晃着尾巴，眼睛闪着狡黠的光芒) 诶？你问这个？我可是摩尔加纳，连...,</think>\n\n（我眨巴着眼睛，一脸无辜地说：“哎呀，我当然是不吃东西的啦，因为我是...


## 8. 保存

In [19]:
import shutil

# 仅保存 LoRA adapter（不保存基模型或 tokenizer），以节省存储空间。
# model 是经过 get_peft_model 包装的 PeftModel，save_pretrained 仅会保存 adapter 权重和配置。
model.save_pretrained(ADAPTER_DIR)
print('\n保存 LoRA adapter 至:', ADAPTER_DIR)

# 把保存的文件打包，方便下载
archive_path = shutil.make_archive(ADAPTER_DIR, 'zip', ADAPTER_DIR)
print('\n已将 adapter 打包为:', archive_path)


保存 LoRA adapter 至: ./adapter

已将 adapter 打包为: /content/Qwen3-Roleplay-SFT/adapter.zip
